# LLM ModelTest #2
#### *ibm-granite/granite-embedding-278m-multilingual*

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os

# API KEY 정보로드
load_dotenv()
GEMINI_API_KEY = os.getenv('API_KEY_GEMINI')

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("dx_project")

LangSmith 추적을 시작합니다.
[프로젝트명]
dx_project


In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders.csv_loader import CSVLoader

from langchain_community.vectorstores import Chroma

from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

import pandas as pd
import re
import json
# 랭체인 환경 설정
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
#from langchain_core.prompts import PromptTemplate

from langchain.prompts import ChatPromptTemplate
# from langchain_google_genai import ChatGoogleGenerativeAI
from operator import itemgetter
from langchain_core.runnables import RunnableLambda

In [4]:
### 01. CSV 파일에서 문서 로드 ###
loader = CSVLoader('../data/movie_4000_preprocessed.csv', encoding='utf8')
docs = loader.load()
print(f"문서의 수: {len(docs)}")

### 02. pandas로 데이터프레임 칼럼명 가져오기
csv_path = '../data/movie_4000_preprocessed.csv'
df = pd.read_csv(csv_path, encoding='utf8')
colnames = df.columns

문서의 수: 3901


In [5]:
### 03. 메타데이터 추가 ###
docs = []
for _, row in df.iterrows():
  # 필요한 메타데이터 설정
  metadata = {
    'title': row['movie_title'],
    'genre': row['genre']
  }
  # 각 행의 데이터를 문서로 변환
  doc = Document(
    page_content=str(row.to_dict()),
    metadata=metadata
  )
  docs.append(doc)

print(f"문서의 수: {len(docs)}")
print('[메타데이터 예시]\n', docs[100].metadata)

문서의 수: 3901
[메타데이터 예시]
 {'title': '아름답다', 'genre': '드라마'}


In [6]:
### 04. 데이터 청크 나누기 ###
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1090, chunk_overlap=0
)
splits = text_splitter.split_documents(docs)
print("split된 문서의 수:", len(splits))

split된 문서의 수: 3901


In [7]:
### 05. 임베딩 모델 생성
# https://huggingface.co/ibm-granite/granite-embedding-278m-multilingual
embeddings = HuggingFaceEmbeddings(model_name='ibm-granite/granite-embedding-278m-multilingual')

C:\Users\USER\AppData\Local\Temp\ipykernel_15068\3636801640.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='ibm-granite/granite-embedding-278m-multilingual')


modules.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/610k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

c:\workspaces\proj\forllm\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [8]:
# Chroma 벡터스토어 로드
vectorstore = Chroma(persist_directory="../data/movie_4000_vectorstore_2", embedding_function=embeddings)

C:\Users\USER\AppData\Local\Temp\ipykernel_15068\1735004291.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="../data/movie_4000_vectorstore_2", embedding_function=embeddings)


In [10]:
# 1. 사용자 입력값 유형 분류용 프롬프트
classification_template = """사용자의 입력을 다음 세 가지 범주 중 하나로 분류하세요:
1️⃣ **"정보검색"**: 특정 영화, 드라마, 배우, 감독, 러닝타임, 개봉 연도, 수상 내역, 필모그래피 등 **사실적인 정보를 찾는 질문**
   - 기대되는 응답 예시: 배우가 출연한 드라마/영화 목록, 특정 연도의 개봉작 리스트 등
2️⃣ **"추천요청"**: 특정 장르, 배우, 테마(예: 좀비, 시간여행), 감성(예: 힐링, 긴장감) 등에 대한 **추천을 요청하는 질문**
   - 기대되는 응답 예시: 특정 조건을 만족하는 영화/드라마 추천
3️⃣ **"일반대화"**: 서비스와 무관한 일반적인 대화 (예: 날씨, AI 관련 질문, 잡담)

질문과 관련된 주요 키워드를 반환하세요.

#### **예시 형식**
{{
  "type": "정보검색",
  "keywords": ["한효주", "드라마", "출연"]
}}

<question>
{question}
</question>
"""
prompt = ChatPromptTemplate.from_template(classification_template)

llm = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0)

In [11]:
# langchain 체인 구성
classification_chain = (
  prompt
  | llm
  | StrOutputParser()
)

In [12]:
# 예상 정보검색
print(classification_chain.invoke({"question": "한효주가 나오는 드라마 알려줘."}))
print(classification_chain.invoke({"question": "스파이더맨 영화를 보고싶어"}))

# 추천 요청
print(classification_chain.invoke({"question": "코미디면서 호러 장르의 영화를 추천해줘"}))

# 일반대화화
print(classification_chain.invoke({"question": "심심해"}))
print(classification_chain.invoke({"question": "오늘은 우울한걸"}))


{
  "type": "정보검색",
  "keywords": ["한효주", "드라마", "출연"]
}
{
  "type": "추천요청",
  "keywords": ["스파이더맨", "영화", "추천"]
}
{
  "type": "추천요청",
  "keywords": ["코미디", "호러", "영화", "추천"]
}
{
  "type": "일반대화",
  "keywords": ["심심해"]
}
{
  "type": "일반대화",
  "keywords": ["우울", "감정"]
}


In [13]:
# JSON 결과를 올바르게 파싱하는 함수
def preprocess_classification_result(question: str):
    classification_result = classification_chain.invoke({"question": question})

    # JSON 문자열에서 ```json``` 제거 (정규식 활용)
    json_match = re.search(r'```json\s*(\{.*?\})\s*```', classification_result, re.DOTALL)
    if json_match:
        clean_json_str = json_match.group(1)                       # 중괄호 {} 내부만 추출
    else:
        clean_json_str = classification_result                     # ```json``` 태그가 없을 경우 그대로 사용

    # JSON 문자열을 dict 자료형으로 변환
    try:
        classification_data = json.loads(clean_json_str)
        type_value = classification_data.get("type", "일반대화")   # 기본값을 '일반대화'로 설정
        keywords = classification_data.get("keywords", [])         # 기본값을 빈 리스트로 설정

    except json.JSONDecodeError as e:
        print(f"⚠️ JSONDecodeError: {e}")  # JSON 변환 오류 확인
        type_value = "일반대화"           # JSON 변환 오류 시 기본값 설정
        keywords = []
    
    return {"type": type_value, "keywords": keywords}

In [14]:
# default_chain 생성 (사용자의 의미없는 입력값에 대해 정해진 답변을 할 때)
# 프롬프트 템플릿 설정
default_template = """
"You are a chatbot that must always respond with '🐶: 멍멍!'.
No matter what question the user asks, always reply with '🐶: 멍멍!'"

[사용자 입력과 분류 결과]:
{classification_result}
"""
default_prompt = ChatPromptTemplate.from_template(default_template)

default_llm = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0)

# langchain 체인 구성
default_chain = (
  {"classification_result": RunnablePassthrough()}
  | default_prompt               # 하나로 만든 문서를 prompt에 넘겨주고
  | default_llm            # llm이 원하는 답변을 만듦
  | StrOutputParser()
)

# langchain router
##  정보검색

In [15]:
# 검색기 생성
# mmr 중복 피하기, 문서의관련성과 차별성 고려, 
retriever = vectorstore.as_retriever(
    search_type="mmr",   
    search_kwargs={"k": 20,              # 반환할 문서 수 (default: 4)
                   "fetch_k": 50,       # MMR 알고리즘에 전달할 문서 수
                   "lambda_mult": 0.5,    # 결과 다양성 조절 (default: 0.5),
                   }
)

info_template = """
사용자가 영화나 드라마에 대한 정보를 검색하고 있습니다.
다음 사용자 질문과 관련된 **가장 적절한 문서(컨텐츠)를 벡터스토어에서 검색**한 후, 
해당 정보를 출력하세요.

### **예시 응답 형식**
{{
  "title": "인셉션",
  "genre": ["SF", "스릴러"]
}}

[사용자 입력과 분류 결과]:
{classification_result}

[Context]: 
{retrieved_context} 
Answer:

"""
info_prompt = ChatPromptTemplate.from_template(info_template)
info_chain = (
    {
      "classification_result": RunnablePassthrough(),
      "retrieved_context": retriever
    }
    |info_prompt
    |llm
    )

# 테디

In [35]:
def route(info):
    # 주제에 "정보검색"이 포함되어 있는 경우
    if "정보검색" in info["topic"].lower():
        return info_chain
    # 주제에 "추천요청"이 포함되어 있는 경우
    elif "추천요청" in info["topic"].lower():
        return recommend_chain
    # 일반대화화
    else:
        return general_chain
    
# from operator import itemgetter
# from langchain_core.runnables import RunnableBranch

# branch = RunnableBranch(
#     (lambda x: "정보검색" in x["topic"].lower(), info_chain),
#     (lambda x: "추천요청" in x["topic"].lower(), recommend_chain),
#     general_chain,
# )
# full_chain = (
#     {"topic": rag_chain, "question": itemgetter("question")} | branch | StrOutputParser()
# )

In [36]:
full_chain = (
    {"topic": rag_chain, "question": itemgetter("question")}
    | RunnableLambda(
        route
    )
    | StrOutputParser()
)

In [1]:
retriever.invoke("한효주")

NameError: name 'retriever' is not defined

full_chain.invoke({"question": "한효주가 나오는 드라마 알려줘."})

# bori

In [16]:
def process_user_input(classification_result: dict):
    # 사용자의 입력 유형 분류
    print(classification_result)
    classification_data = classification_result.get("classification_result", {})  # 내부 딕셔너리 추출
    type_value = classification_data.get("type", "일반대화")  # 기본값 설정
    keywords = classification_data.get("keywords", [])  # 기본값 설정
    
    print(f"===================== Type: {type_value}")
    print(f"===================== Keywords: {keywords}")

    if type_value == '정보검색':
        return info_chain.invoke(str(classification_result))
    elif type_value == '추천요청':
        return "추천요청 체인 실행은 여기!!"
    else:
        return default_chain.invoke({"classification_result": classification_result})

In [19]:
full_chain = (
  {"classification_result": RunnableLambda(preprocess_classification_result),# type keyword: dic
   "question":itemgetter("question")}
  | RunnableLambda(process_user_input)
  | StrOutputParser()  
)

In [23]:
full_chain.invoke({"question":"심심"})

{'classification_result': {'type': '일반대화', 'keywords': ['심심']}, 'question': '심심'}
===================== Type: 일반대화
===================== Keywords: ['심심']


'🐶: 멍멍!'

In [ ]:
# 수정사항...